In [6]:
from datasets.stored_dataframe import CapgMyoDataModule, MyoArmbandDataModule, NinaProDataModule
from models import Classifier, OriginalModel
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import torch
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from torchvision.transforms import Compose, ToTensor, Normalize
from models import LightningXGBClassifier
import os
from definitions import MODELS_FOLDER, ROOT_DIR
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Chinese Experiment

In [14]:
pl.seed_everything(42, workers=True)
lr_lambda=lambda epoch: 1 if epoch<16 else 0.1 if epoch<24 else 0.01
model_capgmyo_chinese = OriginalModel(8, 16, 8)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR
optimizer = torch.optim.RAdam
classifier = Classifier(model_capgmyo_chinese, optimizer=optimizer, lr_scheduler=lr_scheduler, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001}, monitor='val_accuracy', lr_lambda=lr_lambda)
logger_capgmyo_chinese = WandbLogger(project="EMG Armband", name="Chinese CapgMyo")
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000
)
trainer_capgmyo_chinese = pl.Trainer(gpus=-1, max_epochs=28, logger=logger_capgmyo_chinese, accelerator="gpu")

Global seed set to 42
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\loggers\wandb.py:352: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:446: LightningDeprecationWarning: Setting `Trainer(gpus=-1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=-1)` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer_capgmyo_chinese.fit(model=classifier, datamodule=data_module_capgmyo)

C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Science\EMG_Analysis\notebooks\EMG Armband\u4pv5glx\checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | model     | OriginalModel    | 4.6 M 
1 | criterion | CrossEntropyLoss | 0     
-----------------------------------------------
4.6 M     Trainable params
0         Non-trainable params
4.6 M     Total params
18.291    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\pytorch_lightning\trainer\trainer.py:653: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer_capgmyo_chinese.test(model=classifier, datamodule=data_module_capgmyo)

Exception in thread NetStatThr:
Traceback (most recent call last):
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\wandb_run.py", line 185, in check_network_status
    status_response = self._interface.communicate_network_status()
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\interface\interface.py", line 138, in communicate_network_status
    resp = self._communicate_network_status(status)
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\interface\interface_sock.py", line 82, in _communicate_network_status
    data = super()._communicate_network_status(status)
  File "C:\Users\mateu\anaconda3\envs\emg_2\lib\site-packages\wandb\sdk\interface\interface_shared.py", line 410, in _commun

In [10]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'chinese_capgmyo.pt'))

## CapgMyo Experiment

In [ ]:
pl.seed_everything(42, workers=True)
model_capgmyo = OriginalModel(8, 16, 8)
classifier = Classifier(model_capgmyo, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyo")
data_module_capgmyo = CapgMyoDataModule(
    batch_size=1000
)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_capgmyo = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_capgmyo, accelerator="gpu", callbacks=[early_stop_callback])

In [ ]:
trainer_capgmyo.fit(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
trainer_capgmyo.test(model=classifier, datamodule=data_module_capgmyo)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'capgmyo.pt'))

## Ninapro Experiment

In [ ]:
pl.seed_everything(42, workers=True)
model_ninapro = OriginalModel(52, 1, 10)
classifier = Classifier(model_ninapro, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_ninapro = WandbLogger(project="EMG Armband", name="NinaPro")
data_module_ninapro = NinaProDataModule(
    batch_size=1000
)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_ninapro = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_ninapro, accelerator="gpu", callbacks=[early_stop_callback])

In [ ]:
trainer_ninapro.fit(model=classifier, datamodule=data_module_ninapro)

In [ ]:
trainer_ninapro.test(model=classifier, datamodule=data_module_ninapro)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'ninapro.pt'))

## MyoArmband Experiment

In [ ]:
pl.seed_everything(42, workers=True)
model_myoarmband = OriginalModel(7, 1, 8)
classifier = Classifier(model_myoarmband, optim_kwargs={'lr': 0.001, 'weight_decay': 0.0001},
                        monitor='val_accuracy', sched_kwargs={'patience': 4, 'mode': 'max'})
logger_myoarmband = WandbLogger(project="EMG Armband", name="MyoArmband")
transform = Compose([
    ToTensor(),
    Normalize(0, 1)
])
data_module_myoarmband = MyoArmbandDataModule(
    batch_size=10000,
    num_workers=8,
    train_transforms=transform,
    val_transforms=transform,
    test_transforms=transform,
)
early_stop_callback = EarlyStopping(monitor='val_accuracy', patience=7, mode='max')
trainer_myoarmband = pl.Trainer(gpus=-1, max_epochs=100, logger=logger_myoarmband, accelerator="gpu",
                                callbacks=[early_stop_callback])

In [ ]:
trainer_myoarmband.fit(model=classifier, datamodule=data_module_myoarmband)

In [ ]:
trainer_myoarmband.test(model=classifier, datamodule=data_module_myoarmband)

In [ ]:
torch.save(model_capgmyo_chinese.state_dict(), os.path.join(MODELS_FOLDER, 'myoarmband.pt'))

## XGB Experiments

#### XGB CapgMyo

In [ ]:
logger_xgb_capgmyo = WandbLogger(project="EMG Armband", name="CapgMyoXGB")
adjusted_model = model_capgmyo
adjusted_model.model = adjusted_model.model[0:26]
xgb_classifier = LightningXGBClassifier(adjusted_model, 8, monitor='val_accuracy')
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_capgmyo, num_sanity_val_steps=0)
data_module_xgb = CapgMyoDataModule(
    batch_size=1000,
    num_workers=3
)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_capgmyo.json"))

#### XGB Ninapro

In [ ]:
logger_xgb_ninapro = WandbLogger(project="EMG Armband", name="NinaproXGB")
adjusted_model = model_ninapro
adjusted_model.model = adjusted_model.model[0:26]
xgb_classifier = LightningXGBClassifier(adjusted_model, 10)
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_ninapro, num_sanity_val_steps=0)
data_module_xgb = NinaProDataModule(
    batch_size=1000,
    num_workers=3
)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_ninapro.json"))

#### XGB MyoArmband

In [ ]:
logger_xgb_myoarmband = WandbLogger(project="EMG Armband", name="MyoArmbandXGB")
adjusted_model = model_myoarmband
adjusted_model.model = adjusted_model.model[0:26]
xgb_classifier = LightningXGBClassifier(adjusted_model, 8)
trainer_xgb = pl.Trainer(max_epochs=2, accelerator="cpu", logger=logger_xgb_myoarmband, num_sanity_val_steps=0)
data_module_xgb = MyoArmbandDataModule(
    batch_size=1000,
    num_workers=3
)

In [ ]:
trainer_xgb.fit(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
trainer_xgb.test(model=xgb_classifier, datamodule=data_module_xgb)

In [ ]:
xgb_classifier.xgbmodel.save_model(os.path.join(MODELS_FOLDER, "xgb_myoarmband.json"))